In [ ]:
!pip install routingpy pandas openpyxl requests nest-asyncio

In [ ]:
# Distance Calculator for Google Colab - Secure Version
# Installation cell - run this first
"""
!pip install routingpy pandas openpyxl requests nest-asyncio
"""

import pandas as pd
import asyncio
import nest_asyncio
from concurrent.futures import ThreadPoolExecutor
import time
import logging
from datetime import datetime
import os
from typing import Dict, Optional, Union
from google.colab import files
import io
import routingpy

# Enable nested event loops for Colab
nest_asyncio.apply()

class DistanceCalculator:
    def __init__(self):
        # API keys are now loaded from environment variables for security
        self.stop_requested = False
        self.setup_logging()

    def setup_logging(self):
        """Setup logging"""
        self.logger = logging.getLogger('DistanceCalculator')
        self.logger.setLevel(logging.INFO)

        # Clear existing handlers
        for handler in self.logger.handlers[:]:
            self.logger.removeHandler(handler)

        # Create console handler
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        self.logger.addHandler(handler)

    def log(self, level: str, message: str):
        """Log a message"""
        getattr(self.logger, level.lower(), self.logger.info)(message)

    def get_env_api_key(self, provider: str) -> Optional[str]:
        """Get API key from environment variables"""
        env_var_map = {
            "Google Maps": "GOOGLE_MAPS_API_KEY",
            "Mapbox OSRM": "MAPBOX_API_KEY",
            "OpenRouteService": "OPENROUTE_API_KEY",
            "GraphHopper": "GRAPHHOPPER_API_KEY",
            "HERE Maps": "HERE_MAPS_API_KEY"
        }

        env_var = env_var_map.get(provider)
        if env_var:
            return os.getenv(env_var)
        return None

    def setup_api_keys(self):
        """Interactive setup for API keys"""
        print("\n🔐 API Key Setup")
        print("=" * 30)
        print("For security, this version uses environment variables for API keys.")
        print("You have two options:")
        print()
        print("1. Set environment variables in this session:")
        print("   os.environ['GOOGLE_MAPS_API_KEY'] = 'your_key_here'")
        print("   os.environ['MAPBOX_API_KEY'] = 'your_key_here'")
        print("   # etc.")
        print()
        print("2. Enter keys manually when prompted (recommended)")
        print()

        setup_choice = input("Would you like to set up environment variables now? (y/n): ").strip().lower()

        if setup_choice in ['y', 'yes']:
            self._interactive_env_setup()

    def _interactive_env_setup(self):
        """Interactive environment variable setup"""
        providers = {
            "Google Maps": "GOOGLE_MAPS_API_KEY",
            "Mapbox OSRM": "MAPBOX_API_KEY",
            "OpenRouteService": "OPENROUTE_API_KEY",
            "GraphHopper": "GRAPHHOPPER_API_KEY",
            "HERE Maps": "HERE_MAPS_API_KEY"
        }

        print("\n🔧 Setting up environment variables...")
        for provider, env_var in providers.items():
            existing_key = os.getenv(env_var)
            if existing_key:
                print(f"✅ {provider}: Already configured")
                continue

            setup_this = input(f"\nSet up API key for {provider}? (y/n/skip): ").strip().lower()
            if setup_this in ['y', 'yes']:
                api_key = input(f"Enter API key for {provider}: ").strip()
                if api_key:
                    os.environ[env_var] = api_key
                    print(f"✅ {provider} API key configured for this session")
                else:
                    print(f"⚠️ Skipping {provider} - no key entered")
            else:
                print(f"⏭️ Skipping {provider}")

    def select_provider(self) -> str:
        """Interactive provider selection"""
        providers = [
            ("Google Maps", self._get_provider_status("Google Maps")),
            ("OpenRouteService", self._get_provider_status("OpenRouteService")),
            ("GraphHopper", self._get_provider_status("GraphHopper")),
            ("Mapbox OSRM", self._get_provider_status("Mapbox OSRM")),
            ("HERE Maps", self._get_provider_status("HERE Maps")),
            ("Valhalla", "Custom server 🔧"),
            ("OSRM", "Public server 🆓"),
            ("OpenTripPlanner", "Custom server 🔧")
        ]

        print("\n🌐 Select Routing Provider:")
        print("-" * 40)
        for i, (provider, status) in enumerate(providers, 1):
            print(f"{i}. {provider} - {status}")

        while True:
            try:
                choice = input(f"\nEnter choice (1-{len(providers)}): ").strip()
                if choice.isdigit() and 1 <= int(choice) <= len(providers):
                    selected_provider = providers[int(choice) - 1][0]
                    print(f"✅ Selected: {selected_provider}")
                    return selected_provider
                else:
                    print("❌ Invalid choice. Please try again.")
            except (ValueError, KeyboardInterrupt):
                print("❌ Invalid input. Please enter a number.")

    def _get_provider_status(self, provider: str) -> str:
        """Get status for a provider"""
        if self.get_env_api_key(provider):
            return "Configured ✅"
        else:
            return "Requires API key 🔑"

    def get_api_key_if_needed(self, provider: str) -> str:
        """Get API key if required for provider"""
        if provider == "OSRM":
            return None  # No API key needed

        # Check environment variables first
        env_key = self.get_env_api_key(provider)
        if env_key:
            print(f"✅ Using API key from environment variable for {provider}")
            return env_key

        # Ask for API key manually
        print(f"\n🔑 API key needed for {provider}")
        print("Options:")
        print("1. Enter API key manually")
        print("2. Set environment variable and restart")

        choice = input("Choose option (1/2): ").strip()

        if choice == "1":
            api_key = input(f"🔑 Enter API key for {provider}: ").strip()
            if not api_key:
                raise ValueError(f"API key is required for {provider}")
            return api_key
        else:
            env_var_map = {
                "Google Maps": "GOOGLE_MAPS_API_KEY",
                "Mapbox OSRM": "MAPBOX_API_KEY",
                "OpenRouteService": "OPENROUTE_API_KEY",
                "GraphHopper": "GRAPHHOPPER_API_KEY",
                "HERE Maps": "HERE_MAPS_API_KEY"
            }
            env_var = env_var_map.get(provider, f"{provider.upper()}_API_KEY")
            print(f"\n💡 Set this environment variable:")
            print(f"   os.environ['{env_var}'] = 'your_api_key_here'")
            raise ValueError("Please set the environment variable and try again")

    def get_base_url_if_needed(self, provider: str) -> str:
        """Get base URL if required for provider"""
        if provider in ["Valhalla", "OpenTripPlanner"]:
            if provider == "OpenTripPlanner":
                base_url = input(f"🌐 Enter base URL for {provider} (required): ").strip()
                if not base_url:
                    raise ValueError(f"Base URL is required for {provider}")
                return base_url
            else:  # Valhalla
                base_url = input(f"🌐 Enter base URL for {provider} (optional, press Enter to skip): ").strip()
                return base_url if base_url else None
        elif provider == "OSRM":
            base_url = input(f"🌐 Enter custom OSRM server URL (optional, press Enter for public server): ").strip()
            return base_url if base_url else None

        return None

    def load_data_source(self) -> pd.DataFrame:
        """Choose and load data from file upload or URL"""
        print("\n📁 Choose data source:")
        print("1. Upload file from computer")
        print("2. Load from URL")
        print("3. Load from file path (if file is already in Colab)")

        while True:
            choice = input("\nEnter choice (1-3): ").strip()

            if choice == "1":
                return self._upload_file()
            elif choice == "2":
                return self._load_from_url()
            elif choice == "3":
                return self._load_from_path()
            else:
                print("❌ Invalid choice. Please try again.")

    def _upload_file(self) -> pd.DataFrame:
        """Upload and read input file"""
        print("📁 Please upload your CSV or Excel file...")
        uploaded = files.upload()

        if not uploaded:
            raise ValueError("No file uploaded")

        filename = list(uploaded.keys())[0]
        content = uploaded[filename]

        return self._read_file_content(content, filename)

    def _load_from_url(self) -> pd.DataFrame:
        """Load file from URL"""
        import requests

        url = input("🌐 Enter file URL (CSV or Excel): ").strip()
        if not url:
            raise ValueError("URL is required")

        print(f"📥 Downloading file from: {url}")

        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()

            # Determine file type from URL or content type
            if url.lower().endswith('.xlsx') or 'excel' in response.headers.get('content-type', '').lower():
                filename = "downloaded_file.xlsx"
            else:
                filename = "downloaded_file.csv"

            print(f"✅ File downloaded successfully ({len(response.content)} bytes)")

            return self._read_file_content(response.content, filename)

        except requests.RequestException as e:
            raise ValueError(f"Failed to download file from URL: {str(e)}")

    def _load_from_path(self) -> pd.DataFrame:
        """Load file from existing path in Colab"""
        filepath = input("📁 Enter file path: ").strip()
        if not filepath:
            raise ValueError("File path is required")

        if not os.path.exists(filepath):
            raise ValueError(f"File not found: {filepath}")

        print(f"📂 Loading file: {filepath}")

        try:
            if filepath.lower().endswith('.xlsx'):
                df = pd.read_excel(filepath, engine='openpyxl')
            else:
                df = pd.read_csv(filepath, dtype=str)

            return self._validate_and_clean_dataframe(df, os.path.basename(filepath))

        except Exception as e:
            raise ValueError(f"Failed to read file: {str(e)}")

    def _read_file_content(self, content: bytes, filename: str) -> pd.DataFrame:
        """Read file content and return DataFrame"""
        try:
            # Read file based on extension
            if filename.endswith('.xlsx'):
                df = pd.read_excel(io.BytesIO(content), engine='openpyxl')
            elif filename.endswith('.csv'):
                df = pd.read_csv(io.BytesIO(content), dtype=str)
            else:
                # Try to detect format
                try:
                    df = pd.read_csv(io.BytesIO(content), dtype=str)
                    filename = "file.csv"
                except:
                    df = pd.read_excel(io.BytesIO(content), engine='openpyxl')
                    filename = "file.xlsx"

            return self._validate_and_clean_dataframe(df, filename)

        except Exception as e:
            raise ValueError(f"Failed to read file content: {str(e)}")

    def _validate_and_clean_dataframe(self, df: pd.DataFrame, filename: str) -> pd.DataFrame:
        """Validate and clean the loaded DataFrame"""
        # Validate required columns
        required_columns = ['str_lat', 'str_lon', 'des_lat', 'des_lon']
        missing_cols = [col for col in required_columns if col not in df.columns]

        if missing_cols:
            print(f"❌ Missing required columns: {', '.join(missing_cols)}")
            print(f"📋 Available columns: {', '.join(df.columns.tolist())}")
            raise ValueError(f"Missing required columns: {', '.join(missing_cols)}")

        print(f"✅ File loaded successfully: {filename}")
        print(f"📊 Loaded {len(df)} rows")
        print(f"📋 Columns: {', '.join(df.columns.tolist())}")

        # Clean and validate coordinates
        original_rows = len(df)
        df = self._clean_coordinates(df)
        if len(df) < original_rows:
            print(f"⚠️ Removed {original_rows - len(df)} rows with invalid coordinates")

        print(f"✅ {len(df)} valid coordinate pairs ready for processing")
        return df

    def _clean_coordinates(self, df: pd.DataFrame) -> pd.DataFrame:
        """Clean and validate coordinate columns"""
        coord_columns = ['str_lat', 'str_lon', 'des_lat', 'des_lon']
        for col in coord_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            if 'lat' in col:
                df[col] = df[col].clip(-90, 90)
            else:
                df[col] = df[col].clip(-180, 180)
        df = df.dropna(subset=coord_columns)
        return df.reset_index(drop=True)

    def _create_client(self, provider: str, api_key: str = None, base_url: str = None):
        """Create the appropriate routing client"""
        try:
            # Use environment variable key if no custom key provided
            if not api_key:
                api_key = self.get_env_api_key(provider)
                if api_key:
                    print(f"🔐 Using API key from environment for {provider}")

            if provider == "OpenRouteService":
                return routingpy.ORS(api_key=api_key)
            elif provider == "Google Maps":
                return routingpy.Google(api_key=api_key)
            elif provider == "GraphHopper":
                return routingpy.Graphhopper(api_key=api_key)
            elif provider == "HERE Maps":
                return routingpy.HereMaps(api_key=api_key)
            elif provider == "Mapbox OSRM":
                return routingpy.MapboxOSRM(api_key=api_key)
            elif provider == "Valhalla":
                if base_url:
                    return routingpy.Valhalla(base_url=base_url, api_key=api_key)
                else:
                    return routingpy.Valhalla(api_key=api_key)
            elif provider == "OSRM":
                if base_url:
                    return routingpy.OSRM(base_url=base_url)
                else:
                    return routingpy.OSRM()
            elif provider == "OpenTripPlanner":
                if base_url:
                    return routingpy.OpenTripPlannerV2(base_url=base_url, api_key=api_key)
                else:
                    raise ValueError("OpenTripPlanner requires a base URL")
            else:
                raise ValueError(f"Unsupported provider: {provider}")
        except Exception as e:
            self.log("ERROR", f"Failed to create {provider} client: {str(e)}")
            raise

    def _get_profile(self, provider: str) -> str:
        """Get the appropriate profile for the routing provider"""
        profiles = {
            "OpenRouteService": "driving-car",
            "Google Maps": "driving",
            "GraphHopper": "car",
            "HERE Maps": "car",
            "Mapbox OSRM": "driving",
            "OSRM": "driving",
            "Valhalla": "auto",
            "OpenTripPlanner": "CAR"
        }
        return profiles.get(provider, "driving")

    def _get_distance_duration(self, client, provider: str, origin_lat: float, origin_lng: float,
                              dest_lat: float, dest_lng: float) -> Dict[str, str]:
        """Get distance and duration for a single route"""
        try:
            route = client.directions(
                locations=[(origin_lng, origin_lat), (dest_lng, dest_lat)],
                profile=self._get_profile(provider)
            )
            return {
                "distance": f"{route.distance/1000:.2f} km",
                "duration": f"{route.duration/60:.2f} mins"
            }
        except Exception as e:
            raise Exception(f"{provider} request failed: {e}")

    async def _get_distance_duration_async(self, semaphore, client, provider: str,
                                          origin_lat: float, origin_lng: float,
                                          dest_lat: float, dest_lng: float) -> Dict[str, str]:
        """Get distance and duration asynchronously"""
        async with semaphore:
            return await asyncio.to_thread(
                self._get_distance_duration, client, provider, origin_lat, origin_lng, dest_lat, dest_lng
            )

    def process_coordinates(self, df: pd.DataFrame, provider: str = "Google Maps",
                           api_key: str = None, base_url: str = None,
                           batch_size: int = 25, max_workers: int = 5,
                           use_async: bool = True) -> pd.DataFrame:
        """Process coordinates and calculate distances/durations"""

        print(f"\n🚀 Starting distance calculation using {provider}")
        print(f"⚙️ Settings: batch_size={batch_size}, max_workers={max_workers}, async={use_async}")

        # Validation
        if provider != "OSRM" and not api_key and not self.get_env_api_key(provider):
            raise ValueError(f"API key required for {provider}")

        if provider == "OpenTripPlanner" and not base_url:
            raise ValueError("OpenTripPlanner requires a base URL")

        # Create client
        client = self._create_client(provider, api_key, base_url)

        # Prepare output dataframe
        result_df = df.copy()
        result_df["Distance"] = ""
        result_df["Duration"] = ""

        total_rows = len(result_df)
        processed = 0
        errors = 0
        start_time = time.time()

        print(f"📊 Processing {total_rows} coordinate pairs...")

        if use_async:
            # Use create_task instead of asyncio.run() for Colab compatibility
            loop = asyncio.get_event_loop()
            if loop.is_running():
                # We're in a running loop (Colab/Jupyter)
                task = asyncio.create_task(
                    self._process_async(client, provider, result_df, batch_size, max_workers)
                )
                processed, errors = loop.run_until_complete(task)
            else:
                # Standalone environment
                processed, errors = asyncio.run(
                    self._process_async(client, provider, result_df, batch_size, max_workers)
                )
        else:
            # Sync processing with threading
            processed, errors = self._process_sync(
                client, provider, result_df, batch_size, max_workers
            )

        elapsed_time = time.time() - start_time
        success_rate = ((processed - errors) / processed * 100) if processed > 0 else 0

        print(f"\n✅ Processing completed in {elapsed_time:.2f} seconds")
        print(f"📈 Results: {processed-errors} successful, {errors} errors ({success_rate:.1f}% success rate)")

        return result_df

    def _process_sync(self, client, provider: str, df: pd.DataFrame,
                     batch_size: int, max_workers: int) -> tuple:
        """Process synchronously with threading"""
        total_rows = len(df)
        processed = 0
        errors = 0

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            for start_idx in range(0, total_rows, batch_size):
                if self.stop_requested:
                    break

                end_idx = min(start_idx + batch_size, total_rows)
                batch_df = df.iloc[start_idx:end_idx].copy()

                print(f"🔄 Processing batch: rows {start_idx+1}-{end_idx}")

                futures = []
                for idx, row in batch_df.iterrows():
                    future = executor.submit(
                        self._get_distance_duration,
                        client, provider, row["str_lat"], row["str_lon"],
                        row["des_lat"], row["des_lon"]
                    )
                    futures.append((idx, future))

                for idx, future in futures:
                    if self.stop_requested:
                        break

                    try:
                        result = future.result(timeout=30)
                        df.at[idx, "Distance"] = result["distance"]
                        df.at[idx, "Duration"] = result["duration"]
                    except Exception as e:
                        error_msg = f"Error: {str(e)[:50]}..."
                        df.at[idx, "Distance"] = error_msg
                        df.at[idx, "Duration"] = error_msg
                        errors += 1
                        print(f"⚠️ Row {idx+1} failed: {str(e)}")

                    processed += 1

                # Show progress
                progress = (processed / total_rows) * 100
                print(f"📊 Progress: {processed}/{total_rows} ({progress:.1f}%)")
                time.sleep(0.1)

        return processed, errors

    async def _process_async(self, client, provider: str, df: pd.DataFrame,
                           batch_size: int, max_workers: int) -> tuple:
        """Process asynchronously"""
        semaphore = asyncio.Semaphore(max_workers)
        total_rows = len(df)
        processed = 0
        errors = 0

        for start_idx in range(0, total_rows, batch_size):
            if self.stop_requested:
                break

            end_idx = min(start_idx + batch_size, total_rows)
            batch_df = df.iloc[start_idx:end_idx].copy()

            print(f"🔄 Processing async batch: rows {start_idx+1}-{end_idx}")

            tasks = []
            for idx, row in batch_df.iterrows():
                task = self._get_distance_duration_async(
                    semaphore, client, provider, row["str_lat"], row["str_lon"],
                    row["des_lat"], row["des_lon"]
                )
                tasks.append((idx, task))

            for idx, task in tasks:
                if self.stop_requested:
                    break

                try:
                    result = await task
                    df.at[idx, "Distance"] = result["distance"]
                    df.at[idx, "Duration"] = result["duration"]
                except Exception as e:
                    error_msg = f"Error: {str(e)[:50]}..."
                    df.at[idx, "Distance"] = error_msg
                    df.at[idx, "Duration"] = error_msg
                    errors += 1
                    print(f"⚠️ Row {idx+1} failed: {str(e)}")

                processed += 1

            # Show progress
            progress = (processed / total_rows) * 100
            print(f"📊 Progress: {processed}/{total_rows} ({progress:.1f}%)")
            await asyncio.sleep(0.05)

        return processed, errors

    def download_results(self, df: pd.DataFrame, filename: str = None):
        """Download results as Excel file"""
        if filename is None:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f"distance_results_{timestamp}.xlsx"

        # Save to Excel
        df.to_excel(filename, index=False, engine='openpyxl')

        # Download file
        files.download(filename)
        print(f"📥 Results downloaded: {filename}")

# Convenience functions for easy usage
def calculate_distances_interactive():
    """
    Interactive function with provider selection and data source options
    """
    calc = DistanceCalculator()

    # Show API key setup instructions
    calc.setup_api_keys()

    try:
        # Select provider
        provider = calc.select_provider()

        # Get API key if needed
        api_key = calc.get_api_key_if_needed(provider)

        # Get base URL if needed
        base_url = calc.get_base_url_if_needed(provider)

        # Load data source
        df = calc.load_data_source()

        # Get processing settings
        print("\n⚙️ Processing Settings:")
        batch_size = input("Batch size (default=25): ").strip()
        batch_size = int(batch_size) if batch_size.isdigit() else 25

        max_workers = input("Max workers (default=5): ").strip()
        max_workers = int(max_workers) if max_workers.isdigit() else 5

        use_async = input("Use async processing? (y/n, default=y): ").strip().lower()
        use_async = use_async in ['', 'y', 'yes']

        # Process coordinates
        result_df = calc.process_coordinates(
            df, provider=provider, api_key=api_key, base_url=base_url,
            batch_size=batch_size, max_workers=max_workers, use_async=use_async
        )

        # Download results
        custom_filename = input("\nCustom output filename (press Enter for auto): ").strip()
        calc.download_results(result_df, custom_filename if custom_filename else None)

        return result_df

    except KeyboardInterrupt:
        print("\n❌ Operation cancelled by user")
        return None
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        return None

def calculate_distances_simple(provider: str = None, api_key: str = None,
                             batch_size: int = 25, max_workers: int = 5, use_async: bool = True):
    """
    Simple function to upload file, process, and download results in one go

    Args:
        provider: Routing service provider (None for interactive selection)
        api_key: API key (optional if using environment variables)
        batch_size: Number of requests per batch
        max_workers: Maximum concurrent workers
        use_async: Use async processing
    """
    calc = DistanceCalculator()

    # Select provider if not specified
    if provider is None:
        provider = calc.select_provider()
        api_key = calc.get_api_key_if_needed(provider)
        base_url = calc.get_base_url_if_needed(provider)
    else:
        base_url = None

    # Upload file
    df = calc.load_data_source()

    # Process coordinates
    result_df = calc.process_coordinates(
        df, provider=provider, api_key=api_key, base_url=base_url,
        batch_size=batch_size, max_workers=max_workers, use_async=use_async
    )

    # Download results
    calc.download_results(result_df)

    return result_df

# Helper function to set up environment variables
def setup_environment_variables():
    """
    Helper function to set up API keys as environment variables
    """
    print("🔧 Environment Variable Setup")
    print("=" * 35)
    print("Set your API keys as environment variables for secure usage:")
    print()

    providers = {
        "Google Maps": "GOOGLE_MAPS_API_KEY",
        "Mapbox OSRM": "MAPBOX_API_KEY",
        "OpenRouteService": "OPENROUTE_API_KEY",
        "GraphHopper": "GRAPHHOPPER_API_KEY",
        "HERE Maps": "HERE_MAPS_API_KEY"
    }

    print("📝 Example usage:")
    for provider, env_var in providers.items():
        print(f"   os.environ['{env_var}'] = 'your_{provider.lower().replace(' ', '_')}_api_key'")

    print()
    print("💡 Or use this interactive setup:")

    setup_choice = input("Set up API keys interactively? (y/n): ").strip().lower()
    if setup_choice in ['y', 'yes']:
        calc = DistanceCalculator()
        calc._interactive_env_setup()
        print("\n✅ Environment variables configured for this session!")

    return providers

# Initialize
print("🛣️ Distance Calculator for Google Colab - SECURE VERSION")
print("=" * 60)
print("\n🔒 SECURITY UPDATE: API keys are now handled securely!")
print("✅ No more hardcoded API keys in source code")
print("🔐 Uses environment variables for API key storage")
print("\n💡 Quick Setup:")
print("1. setup_environment_variables()  # Set up your API keys")
print("2. result_df = calculate_distances_interactive()  # Run the calculator")
print("\n🆘 Need help? The interactive functions will guide you!")

In [ ]:
setup_environment_variables()

In [ ]:
result_df = calculate_distances_interactive()